In [2]:
import pandas as pd
import numpy as np

#df = pd.read_csv(r"C:\Users\kavya\Desktop\Kavya\Intelihub\client\HERC\Latest\data\model_ready_2.csv")

In [3]:
df = pd.read_csv(r"C:\Users\HP\Documents\Load_Forecasting_using_Fbprophet\Final Files fbprophet\Combined_Data_for_model_final_2.csv")

In [4]:
del df['Unnamed: 0']

#### Extracting Cal Features

In [5]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
df = df[df['Date']>='2018-01-01']

In [7]:
df1 = df[df['Date'] < '2020-03-01']

In [8]:
df2 = df[df['Date'] > '2020-04-30']

In [9]:
df3 = df1.append(df2)

In [10]:
df_covidless = df3

In [11]:
df_covidless.to_csv('df_covidless.csv')

In [12]:
df = df_covidless

In [13]:
df['month'] = pd.to_datetime(df['Date']).dt.month
df['weekday'] = pd.to_datetime(df['Date']).dt.weekday
df['hour'] = pd.to_datetime(df['Date']).dt.hour
df['hour_part'] = pd.to_datetime(df['Date']).dt.minute
df['year'] = pd.to_datetime(df['Date']).dt.year
df['day'] = pd.to_datetime(df['Date']).dt.day

# remove outliers in windspeed - checked the max windspeed India recorded - it is around 100
df = df[df['windspeed']<=100]

<AxesSubplot:>

In [14]:
df['winddir'].hist()

In [15]:
def get_season(TimeSeries):
    
    season_list = []
    
    year_list = []
    month_list = []
    for time in TimeSeries:
        year = str(time)[:4]
        month = str(time)[5:7]
        year_list.append(year)
        month_list.append(month) 
       
    for m in month_list:
        if m=='01' or m=='02' or m=='12':
            season_list.append(0)   # Winter
        elif m=='03' or m=='04' or m=='05':
            season_list.append(1) #Spring-Summer
        elif m == '06' or m=='07':
            season_list.append(2) #Paddy/Monsoon
        elif m=='08' or m=='09':
            season_list.append(3) #Late Monsoon
        elif m=='10' or m=='11':
            season_list.append(4) #Autumn
            
    #season = pd.Series(season_list)
    return season_list

In [16]:
season = get_season(df['Date'])

In [17]:
df['Season'] = season

C:\Users\HP\AppData\Local\Temp/ipykernel_364/3332413871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season'] = season


In [18]:
from sklearn import metrics
import numpy as np
from sklearn.model_selection import StratifiedKFold
import itertools
# from skopt import BayesSearchCV as bayes_opt

In [19]:
# check for 0 values and remove
df = df[df['Haryana Load (MW)']>0]

In [20]:
# Train Data
train = df[df['Date']<="2021-06-01"]

In [21]:
# Test Data
test = df[df['Date']>"2021-06-01"]

In [22]:
x_train = train[['month', 'hour', 'weekday', 'temp',
       'humidity', 'winddir', 'visibility', 'cloudcover',
       'hour_part', 'day', 'Season']]
y_train = train[['Haryana Load (MW)']]

In [23]:
x_test = test[['month', 'hour', 'weekday', 'temp',
       'humidity', 'winddir', 'visibility', 'cloudcover',
       'hour_part', 'day', 'Season']]
y_test = test[['Haryana Load (MW)']]

In [24]:
x_test.describe()

,month,hour,weekday,temp,humidity,winddir,visibility,cloudcover,hour_part,day,Season
count,20543.000000,20543.000000,20543.000000,20543.000000,20543.000000,20543.000000,20543.000000,20543.000000,20543.000000,20543.000000,20543.000000
mean,9.004819,11.500560,2.990751,26.290746,72.787525,144.249331,2.849107,44.337088,22.501095,15.790440,2.565448
std,1.998813,6.922058,1.988310,7.092862,19.693465,112.560636,1.105441,35.743741,16.770591,8.827231,1.298122
min,6.000000,0.000000,0.000000,5.000000,19.570000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,7.000000,6.000000,1.000000,21.400000,58.110000,40.000000,2.000000,0.000000,15.000000,8.000000,2.000000
50%,9.000000,12.000000,3.000000,28.000000,77.140000,120.000000,3.000000,50.000000,30.000000,16.000000,3.000000
75%,11.000000,17.500000,5.000000,31.000000,88.920000,260.000000,4.000000,86.500000,37.500000,23.000000,4.000000
max,12.000000,23.000000,6.000000,43.000000,100.000000,360.000000,9.000000,100.000000,45.000000,31.000000,4.000000


In [25]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [26]:
model = xgb.XGBRegressor()
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [27]:
preds = model.predict(x_test)

In [28]:
y_test['pred'] = preds

C:\Users\HP\AppData\Local\Temp/ipykernel_364/2190438978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['pred'] = preds


In [29]:
from sklearn.metrics import r2_score
r2_score(y_test['Haryana Load (MW)'],y_test['pred'])

0.7239651410708295

##### R2 = 0.727

In [30]:
from sklearn.ensemble import RandomForestRegressor

In [31]:
regr = RandomForestRegressor()

In [32]:
regr.fit(x_train, y_train)

C:\Users\HP\AppData\Local\Temp/ipykernel_364/1990009532.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(x_train, y_train)


RandomForestRegressor()

In [33]:
preds = model.predict(x_test)

In [34]:
y_test['pred'] = preds

C:\Users\HP\AppData\Local\Temp/ipykernel_364/2190438978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['pred'] = preds


In [35]:
from sklearn.metrics import r2_score
r2_score(y_test['Haryana Load (MW)'],y_test['pred'])

0.7239651410708295

##### Bucket Analysis

In [42]:
train_predictions = model.predict(x_train)
y_train = y_train.reset_index().drop(columns='index')
y_train_with_error = y_train.join(pd.Series(train_predictions, name='preds'))
y_train_with_error['Error'] = y_train_with_error['Haryana Load (MW)'] - y_train_with_error['preds']
y_train_with_error['Percent Error'] = y_train_with_error['Error']*100/y_train_with_error['Haryana Load (MW)']
y_errorful_buckets = y_train_with_error[np.abs(y_train_with_error['Percent Error']) > 8]
x_train = x_train.reset_index().drop(columns='index')
x_train_new = x_train.drop(index = y_errorful_buckets.index)
y_train_new = y_train.drop(index = y_errorful_buckets.index)

In [43]:
model2 = RandomForestRegressor()
model2.fit(x_train_new, y_train_new)
preds2 = model2.predict(x_test)

C:\Users\HP\AppData\Local\Temp/ipykernel_364/3042317212.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model2.fit(x_train_new, y_train_new)


In [44]:

from sklearn.metrics import r2_score

r2s = r2_score(y_test['Haryana Load (MW)'], preds2)

from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test['Haryana Load (MW)'], preds2))

print(r2s)
print(rmse)

0.7266323456064784
1009.3012371150407
